In [ ]:
# Install requirements
!pip install --quiet openai groq jsonschema

import os
import json
from openai import OpenAI   # Groq is OpenAI-compatible
from jsonschema import validate, ValidationError

# ------------------------------
# 🔑 Set your API key here
# ------------------------------
os.environ["GROQ_API_KEY"] = "use ur known api_key"  # <-- Replace with your actual key!

# Instantiate client for Groq
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"]
)

# ------------------------------
# Schema for extracted info
# ------------------------------
info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"},
    },
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False,
}

# ------------------------------
# Sample user chats
# ------------------------------
sample_chats = [
    "Hello, my name is Alice Brown. I'm 29, live in Delhi. You can reach me at alice@example.com or 9812312345.",
    "John Doe, age 34, from Mumbai. Email: johndoe@gmail.com, phone 9004400440.",
    "This is Priya Singh from Bangalore, 24 years old. Email: priya24@yahoo.com, phone: 8123456789."
]

# ------------------------------
# Extraction + Validation
# ------------------------------
extracted_list = []
for chat in sample_chats:
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",   # ✅ Supported Groq model
        messages=[
            {"role": "system", "content": "Extract structured user info as JSON strictly matching the schema."},
            {"role": "user", "content": chat}
        ],
        response_format={"type": "json_object"}  # ✅ Force JSON output
    )

    # Parse JSON response safely
    # Check if tool_calls exist and extract arguments
    if response.choices[0].message.tool_calls:
        args_json = response.choices[0].message.tool_calls[0].function.arguments
        extracted = json.loads(args_json)
    else:
        # Fallback for responses without tool_calls (though with json_object format, it should have content)
        # This part might need adjustment based on exact API response structure
        try:
             extracted = json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
             print(f"Warning: Could not decode JSON from response content for chat: {chat}")
             extracted = {} # Or handle as appropriate


    extracted_list.append(extracted)

    # Validate with schema
    try:
        validate(instance=extracted, schema=info_schema)
        print(f"✅ Valid Output:\n{json.dumps(extracted, indent=2)}\n")
    except ValidationError as e:
        print(f"❌ Invalid Output: {e.message}\nData: {extracted}\n")
    except Exception as e:
        print(f"An unexpected error occurred during validation: {e}\nData: {extracted}\n")


print("Final Extracted List:", json.dumps(extracted_list, indent=2))

❌ Invalid Output: 'email' is a required property
Data: {'name': 'Alice Brown', 'age': 29, 'location': 'Delhi', 'contact': {'email': 'alice@example.com', 'phoneNumber': '9812312345'}}

✅ Valid Output:
{
  "name": "John Doe",
  "age": 34,
  "location": "Mumbai",
  "email": "johndoe@gmail.com",
  "phone": "9004400440"
}

✅ Valid Output:
{
  "name": "Priya Singh",
  "age": 24,
  "location": "Bangalore",
  "email": "priya24@yahoo.com",
  "phone": "8123456789"
}

Final Extracted List: [
  {
    "name": "Alice Brown",
    "age": 29,
    "location": "Delhi",
    "contact": {
      "email": "alice@example.com",
      "phoneNumber": "9812312345"
    }
  },
  {
    "name": "John Doe",
    "age": 34,
    "location": "Mumbai",
    "email": "johndoe@gmail.com",
    "phone": "9004400440"
  },
  {
    "name": "Priya Singh",
    "age": 24,
    "location": "Bangalore",
    "email": "priya24@yahoo.com",
    "phone": "8123456789"
  }
]
